# Import ness. lib

In [2]:
%pip install opencv-python

     |████████████████████████████████| 33.1 MB 45.3 MB/s eta 0:00:01
You should consider upgrading via the '/Users/jirayuwat/Desktop/GAN-indiv-study/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
import cv2
import os
import matplotlib.pyplot as plt
import re

# Create extract function

In [44]:
def fetch_and_show_images(clip_path, 
                          save_dir, 
                          capture_interval=20
                          ):
    """
    Fetch images from a clip, save as individual files, then visualize using subplots.
    
    Parameters:
    - clip_path (str): Path to the video clip.
    - save_dir (str): Folder to save the fetched images.
    - capture_interval (int): Span to fetch images.
    """
    clip_name = re.findall("[^/]+/([^.]*).*", clip_path)[-1]

    # Verify if save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # Initialize video reader
    clip_reader = cv2.VideoCapture(clip_path)
    frame_total = int(clip_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(clip_reader.get(cv2.CAP_PROP_FPS))
    duration = frame_total // fps


    print(f"Extracting {clip_name}")
    print(f"Frames available in the clip: {frame_total} ({duration} sec.)")
    print(f"Fetching every {capture_interval} second or {fps * capture_interval} frames")
    capture_interval *= fps

    current_frame = 0
    fetched_count = 0
    image_list = []

    while True:
        status, image = clip_reader.read()

        # If reading frame fails, exit the loop
        if not status:
            break
        
        # Switch from BGR (OpenCV default) to RGB for visualization with matplotlib
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if current_frame % capture_interval == 0:
            image_filename = f"{clip_name}-{current_frame/fps:.2f}.png"
            image_filepath = os.path.join(save_dir, image_filename)
            
            cv2.imwrite(image_filepath, image)
            image_list.append(image_rgb)
            fetched_count += 1
            print(f"Fetched frame {current_frame}({current_frame / fps}) as {image_filename}")

        current_frame += 1

    clip_reader.release()

    # Display images using subplots
    subplot_rows = len(image_list) // 2 + len(image_list) % 2
    figure, axes = plt.subplots(subplot_rows, 2, figsize=(10, 10))

    for idx, axis in enumerate(axes.ravel()):
        if idx < len(image_list):
            axis.imshow(image_list[idx])
            axis.set_title(f"Snapshot at {idx * capture_interval}")
            axis.axis('off')
        else:
            axis.axis('off')

    print(f"Process complete! Fetched {fetched_count} frames from the clip.")

In [ ]:
fetch_and_show_images(clip_path='./2021_0607_184742_013.MOV',
                      save_dir='./image_pool',
                      capture_interval=10)